# RMSX Demonstration

This notebook includes everything you need to run **RMSX** and **Flipbook**. Just run each cell. You are encouraged to try different parameters and use your own trajectories directly within this notebook or by making a copy of it.


We'll cover:

1. Environment setup
2. Loading demo input files
3. Single-chain RMSX
4. Multi-chain RMSX
5. FlipBook snapshots (optional)
6. Interpreting results

> **Note:** Adjust file paths as needed for your machine. We reccomend not editing the first two cells, though.

## 1) Environment Setup

- **R** — used for heatmaps and RMSD/RMSF plots via `Rscript`.
- **ChimeraX** (optional) — used for FlipBook image generation.


### Quick Start

In [ ]:
# One-click install for this kernel (no sudo; no Plotly required)
import sys, os, site, tempfile, urllib.request, subprocess, shutil

def _ensure_pip_in_kernel():
    try:
        import pip  # noqa: F401
        return
    except Exception:
        pass
    # Bootstrap pip into THIS kernel if it's missing
    print("Bootstrapping pip into this Jupyter kernel…")
    with tempfile.TemporaryDirectory() as td:
        gp = os.path.join(td, "get-pip.py")
        urllib.request.urlretrieve("https://bootstrap.pypa.io/get-pip.py", gp)
        subprocess.check_call([sys.executable, gp, "--user"])
    # Make sure user-site is importable in this process
    us = site.getusersitepackages()
    if us not in sys.path:
        sys.path.append(us)

_ensure_pip_in_kernel()

# Use Jupyter's %pip so installs target THIS kernel

%pip install -U pip wheel setuptools
%pip install -U "numpy>=1.26,<2.0" "pandas>=2.2,<2.3" "MDAnalysis>=2.6,<3" "scipy>=1.10" "plotly>=5"
%pip install -U "git+https://github.com/AntunesLab/rmsx.git@main" plotly>=5

# Optional: show Rscript (for plots) if available
import subprocess
RSCRIPT = os.environ.get("RSCRIPT", "Rscript")
try:
    out = subprocess.run([RSCRIPT, "-e", "cat(R.version.string)"], capture_output=True, text=True)
    print(out.stdout or out.stderr or "Rscript OK")
except FileNotFoundError:
    print("⚠️ Rscript not found. Heatmap/RMSD/RMSF plots will be skipped until R is installed or RSCRIPT is set.")




*Windows users: if R isn’t on PATH, set `RSCRIPT` before running:*

In [ ]:
import os
# os.environ["RSCRIPT"] = r"C:\\Program Files\\R\\R-4.4.1\\bin\\Rscript.exe"  # <- edit if needed
!pwd

## 2) Load Demo Input Files

These come with the package as `test_files`. Rather than editing these we recocomend adding your paths in later cells. 

In [ ]:
# Imports & locate sample data
from pathlib import Path
import os, subprocess

from rmsx import run_rmsx, run_rmsx_flipbook, all_chain_rmsx, run_shift_flipbook
import rmsx

pkg_dir = Path(rmsx.__file__).resolve().parent

# Optional: set this to use your own dataset folder
# os.environ["RMSX_TEST_DIR"] = "/path/to/test_files"
env_override = os.environ.get("RMSX_TEST_DIR")

candidates = []
if env_override:
    candidates.append(Path(env_override))

candidates += [
    Path.cwd() / "test_files",              # next to this notebook
    Path.cwd() / "rmsx" / "test_files",     # if you cloned into ./rmsx
    pkg_dir / "test_files",                 # packaged tests (if included)
    pkg_dir.parent / "test_files",          # repo layout when running from source
]

test_dir = next((p for p in candidates if p.exists()), None)

# Last resort: clone repo to fetch sample data (does not affect installed package)
if not test_dir:
    REPO_URL = "https://github.com/AntunesLab/rmsx.git"
    REPO_DIR = Path.cwd() / "rmsx"
    if not REPO_DIR.exists():
        print("Cloning RMSX repo to retrieve sample data…")
        subprocess.check_call(["git", "clone", REPO_URL, str(REPO_DIR)])
    test_dir = REPO_DIR / "test_files"
    if not test_dir.exists():
        raise FileNotFoundError("Couldn't locate test_files after cloning. Please report this issue.")

# Single-chain demo (Ubiquitin)
pdb_file   = (test_dir / "1UBQ.pdb").as_posix()
dcd_file   = (test_dir / "mon_sys.dcd").as_posix()
output_dir = (test_dir / "example_uqb").as_posix()

# Multi-chain demo (protease) — files live directly under test_files/
pdb_file_multi   = (test_dir / "protease_backbone.pdb").as_posix()
traj_file_multi  = (test_dir / "short_protease_backbone.dcd").as_posix()
output_dir_multi = (test_dir / "protease").as_posix()

print("Single-chain inputs:")
print("  PDB:", pdb_file)
print("  DCD:", dcd_file)
print("  out:", output_dir)

print("\nMulti-chain inputs:")
print("  PDB:", pdb_file_multi)
print("  DCD:", traj_file_multi)
print("  out:", output_dir_multi)

## 3) Single-Chain RMSX

`run_rmsx` computes per-slice RMSX and generates:
- RMSX heatmap
- RMSD time series
- RMSF per-residue

Edit `palette` and `chain_sele` as needed. (For the included Ubiquitin demo, the chain is `"7"`.)

In [ ]:
from rmsx import *

run_rmsx(
    topology_file=pdb_file,        # PDB or topology file
    trajectory_file=dcd_file,      # Trajectory file
    output_dir=output_dir,         # Where to save results
    num_slices=9,                  # Select number of slices and it will autogenerate slice size 
    slice_size=None,               # OR: select slice size and it will choose number of slices 
    rscript_executable=os.environ.get('RSCRIPT', 'Rscript'),
    verbose=False,                 
    interpolate=False,             # Interpolate between heatmap cells (use with caution)
    triple=True,                   # Generate RMSX, RMSD, and RMSF plots
    overwrite=True,                # Automatically overwrite old directory
    palette="mako",                # Try other colors: viridis, turbo, magma or plasma etc.
    chain_sele="7",                # Set to your chain ID or None to be prompted with available chain 
    start_frame=0,                 # Default to the first frame of the simulation
    end_frame=None                 # Default to None (last frame of simulation)
)
print("Done. Outputs in:", output_dir)

## 4) Multi-Chain RMSX

`all_chain_rmsx` runs RMSX for **every** chain in the topology. If you set `sync_color_scale=True`,
all plots share a consistent colorbar range.

In [ ]:

all_chain_rmsx(
    topology_file=pdb_file,
    trajectory_file=dcd_file,
    output_dir=output_dir,
    num_slices=12,
    slice_size=None,
    rscript_executable=os.environ.get('RSCRIPT', 'Rscript'),
    verbose=False,
    interpolate=False,
    triple=True,
    overwrite=True,
    palette="turbo",
    start_frame=0,
    end_frame=None,
    sync_color_scale=True           # Useful when chains have very different RMSX values, True will force the same color key/scale to be used
)

## 5) FlipBook (optional)

FlipBook maps RMSX values (stored in the PDB B-factor column) onto 3D structures and lays out snapshots side-by-side.

In [ ]:
run_rmsx_flipbook(
    topology_file=pdb_file_multi,
    trajectory_file=traj_file_multi,
    output_dir=output_dir_multi,
    num_slices=9,
    slice_size=None,
    rscript_executable=os.environ.get('RSCRIPT', 'Rscript'),
    verbose=False,
    interpolate=False,
    triple=True,
    overwrite=True,
    palette="turbo",         # automatically matches color palette for heatmap and chimeraX structures 
    spacingFactor="0.9",     # how much space between the snapshots; 1 is default
    start_frame=0,
    end_frame=None
)

### 5.1 Display the latest FlipBook image (if generated)
If you don't like the orientation, try adjusting it in ChimeraX and rerunning the save command (see the ChimeraX log for details)

In [ ]:
import glob
from IPython.display import Image, display
imgs = sorted(glob.glob(os.path.join(output_dir_multi, "combined", "*.png")))
if imgs:
    display(Image(filename=imgs[-1]))
else:
    print("No flipbook images found in:", os.path.join(output_dir_multi, "combined"))

## 6) Interpreting Results

If you want the flanking plots, `triple=True`, RMSX produces:
- **RMSX heatmap** (residue × time slice)
- **RMSD** over time 
- **RMSF** per residue

**Questions to ask:**
1. Which residues/regions show the most flexibility?
2. When do flexibility spikes occur (early/mid/late)?
3. Do multiple intervals of high fluctuation appear?
4. How do these align with functional domains or events (e.g., binding, domain motion)?

---
### Conclusions

You’ve seen how to:
1. Install and configure **RMSX** and **Flipbook**
2. Run single- and multi-chain analyses
3. Generate FlipBook snapshots (optional)
4. Interpret RMSX plots (when/where the biggest changes happen)

For more details, visit the GitHub repo or open an issue with questions.

Feel free to email the primary developer Finn Beruldsen at fpberuld@cougarnet.uh.edu if you need anything else.


### Acknowledgements
We thank the following beta tests for their valuable feedback on this tutorial:
- Mason Kretiv — Texas A&M University
- Helen Ashdown — NIH Biowulf
- John Miles — University of Missouri